# Download necessary files

This notebook downloads files necessary to replicate the analysis in Depsky et al. 2022.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import shutil
import tarfile
from io import BytesIO
from pathlib import Path
from tempfile import NamedTemporaryFile, TemporaryDirectory
from zipfile import ZipFile

import requests
from cartopy.io import shapereader
from shared import (
    DIR_GADM,
    FS,
    PATH_BORDERS,
    PATH_COASTLINES,
    PATH_SLIIDERS_ECON,
    PATH_SLIIDERS_SLR,
)

## Plotting shapefiles

These are necessary to run the [pyCIAM-results-figures.ipynb](./pyCIAM-results-figures.ipynb) notebook, but are not directly used in the model execution.

### Natural Earth Country Boundaries

In [ ]:
def copy_to_dst(glob, dst):
    for f in glob:
        if not (dst / f.name).is_file():
            shutil.copy(f, dst)

In [ ]:
PATH_BORDERS.parent.mkdir(exist_ok=True, parents=True)
PATH_COASTLINES.parent.mkdir(exist_ok=True, parents=True)
fname_boundary = Path(
    shapereader.natural_earth(
        resolution="50m", category="cultural", name="admin_0_boundary_lines_land"
    )
)
fname_coastlines = Path(
    shapereader.natural_earth(resolution="110m", category="physical", name="coastline")
)
copy_to_dst(fname_boundary.parent.glob(fname_boundary.stem + "*"), PATH_BORDERS.parent)
copy_to_dst(
    fname_coastlines.parent.glob(fname_coastlines.stem + "*"), PATH_COASTLINES.parent
)

### GADM

In [ ]:
DIR_GADM.mkdir(exist_ok=True, parents=True)

with TemporaryDirectory() as tmpdir:
    with ZipFile(
        BytesIO(
            requests.get(
                "https://biogeo.ucdavis.edu/data/gadm3.6/gadm36_levels_shp.zip"
            ).content
        )
    ) as zipfile:
        zipfile.extractall(tmpdir)
    for i in [0, 1]:
        copy_to_dst(Path(tmpdir).glob(f"gadm36_{i}*"), DIR_GADM)

## SLIIDERS

SLIIDERS-SLR is a large file (17 GB) so this may take awhile. Users may download these datasets [via GUI](https://zenodo.org/record/6426191#.YlRGPC-B1pQ) as an alternative option

In [ ]:
def download_sliiders(src, dst):
    with NamedTemporaryFile() as tmpfile:
        tmpfile.write(requests.get(src).content)
        with TemporaryDirectory() as tmpdir:
            with tarfile.open(tmpfile.name, "r:gz") as tar:
                tar.extractall(tmpdir)
            FS.upload(tmpdir, dst, recursive=True)

In [ ]:
for kind, dst in [("econ", PATH_SLIIDERS_ECON.root), ("slr", PATH_SLIIDERS_SLR.root)]:
    download_sliiders(
        f"https://zenodo.org/record/6449231/files/sliiders-{kind}.zarr.tar.gz?download=1",
        dst,
    )